In [4]:
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe

In [5]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

In [7]:
lims_query = "SELECT donors.full_genotype, proj.code, \
COUNT(CASE WHEN specimens.patched_cell_container NOTNULL THEN 1 END) AS tubes \
FROM donors JOIN specimens ON specimens.donor_id = donors.id \
JOIN projects proj ON specimens.project_id = proj.id \
JOIN ephys_roi_results err ON specimens.ephys_roi_result_id = err.id \
WHERE err.recording_date BETWEEN '2018-05-07 00:00:00' AND '2018-05-13 24:00:00' AND \
proj.code <> 'hIVSCC-MET' \
GROUP BY donors.full_genotype, proj.code"

lims_df = get_lims_dataframe(lims_query)
print lims_df

         code  tubes                                      full_genotype
0  mIVSCC-MET      1                   Slc17a8-iCre/wt;Ai14(RCL-tdT)/wt
1        T301      1                   Slc17a8-iCre/wt;Ai14(RCL-tdT)/wt
2  mIVSCC-MET     17                   Vip-IRES-Cre/wt;Ai14(RCL-tdT)/wt
3  mIVSCC-MET      7                 Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt
4  mIVSCC-MET      3                   Sst-IRES-Cre/wt;Ai14(RCL-tdT)/wt
5  mIVSCC-MET      4  Htr3a-Cre_NO152/wt;Sst-IRES-FlpO/wt;Ai65(RCFL-...
6  mIVSCC-MET      2  Vipr2-IRES2-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai65(RC...
7  mIVSCC-MET      1               Slc32a1-IRES-Cre/wt;Ai14(RCL-tdT)/wt
